In [1]:
import pandas as pd
import sqlite3 
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import inspect
from sqlalchemy import MetaData
import sqlite3
import datetime

Extract

In [2]:
con = sqlite3.connect("Northwind_large.sqlite")

TRANSFORM

In [12]:
df_custom = pd.read_sql_query(
    """
   SELECT Id AS Id_customer,
       CompanyName,
       ContactName,
       ContactTitle,
       Address,
       City,
       Region,
       ifnull(PostalCode,'No Asignado') as PostalCode,
       replace(replace(Country,"UK","United Kingdom"),"USA","The United States of America")  as Country,
       Phone,
       ifnull(Fax,'No asignado') as Fax
FROM Customer;
    """
    ,con)
df_custom.head(5)

,Id_customer,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,Western Europe,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,Central America,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,Central America,05023,Mexico,(5) 555-3932,No asignado
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,British Isles,WA1 1DP,United Kingdom,(171) 555-7788,(171) 555-6750
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,Northern Europe,S-958 22,Sweden,0921-12 34 65,0921-12 34 67


In [13]:
df_employee = pd.read_sql_query(
    """
SELECT e.Id AS Id_employe,
       e.Lastname,
       e.FirstName,
       e.Title,
       replace(replace(replace(replace(e.TitleOfCourtesy,"Dr","Doctor"),"Ms","Lady"),"Mr","Mister"),"Mrs"," Missus") as TitleOfCourtesy,
       e.BirthDate,
       e.HireDate,
       e.Address,
       e.City,
       e.Region,
       e.PostalCode,
       replace(replace(e.Country,"UK","United Kingdom"),"USA","The United States of America") as Country,
       e.HomePhone,
       e.Extension,
       ifnull(e.Photo,'No Asignado') as Photo,
       e.Notes,
       e.PhotoPath
FROM employee AS e;
    """
    ,con)
df_employee.head(5)

,Id_employe,LastName,FirstName,Title,TitleOfCourtesy,BirthDate,HireDate,Address,City,Region,PostalCode,Country,HomePhone,Extension,Photo,Notes,PhotoPath
0,1,Davolio,Nancy,Sales Representative,Lady.,1980-12-08,2024-05-01,507 - 20th Ave. E. Apt. 2A,Seattle,North America,98122,The United States of America,(206) 555-9857,5467,No Asignado,Education includes a BA in psychology from Col...,http://accweb/emmployees/davolio.bmp
1,2,Fuller,Andrew,"Vice President, Sales",Doctor.,1984-02-19,2024-08-14,908 W. Capital Way,Tacoma,North America,98401,The United States of America,(206) 555-9482,3457,No Asignado,Andrew received his BTS commercial in 1974 and...,http://accweb/emmployees/fuller.bmp
2,3,Leverling,Janet,Sales Representative,Lady.,1995-08-30,2024-04-01,722 Moss Bay Blvd.,Kirkland,North America,98033,The United States of America,(206) 555-3412,3355,No Asignado,Janet has a BS degree in chemistry from Boston...,http://accweb/emmployees/leverling.bmp
3,4,Peacock,Margaret,Sales Representative,Misters.,1969-09-19,2025-05-03,4110 Old Redmond Rd.,Redmond,North America,98052,The United States of America,(206) 555-8122,5176,No Asignado,Margaret holds a BA in English literature from...,http://accweb/emmployees/peacock.bmp
4,5,Buchanan,Steven,Sales Manager,Mister.,1987-03-04,2025-10-17,14 Garrett Hill,London,British Isles,SW1 8JR,United Kingdom,(71) 555-4848,3453,No Asignado,Steven Buchanan graduated from St. Andrews Uni...,http://accweb/emmployees/buchanan.bmp


In [14]:
df_location = pd.read_sql_query(
    """
SELECT [order].Id AS Id_location,
                [order].ShipAddress AS Address,
                [order].ShipCity AS City,
                [order].ShipRegion  AS Region,
                ifnull([order].ShipPostalCode, 'No Asignado') AS PostalCode,
                [replace]([replace]([order].ShipCountry, "UK", "Unit Kingdom"), "USA", "The United States of America") AS Country
  FROM [order];
    """
    ,con)
df_location.head(5)

,Id_location,Address,City,Region,PostalCode,Country
0,10248,59 rue de l'Abbaye,Reims,Western Europe,51100,France
1,10249,Luisenstr. 48,Münster,Western Europe,44087,Germany
2,10250,"Rua do Paço, 67",Rio de Janeiro,South America,05454-876,Brazil
3,10251,"2, rue du Commerce",Lyon,Western Europe,69004,France
4,10252,"Boulevard Tirou, 255",Charleroi,Western Europe,B-6000,Belgium


In [15]:
df_shipper = pd.read_sql_query(
    """

SELECT Id AS Id_shipper,
       CompanyName,
       Phone
  FROM shipper;
    """
    ,con)
df_shipper.head(5)

,Id_shipper,CompanyName,Phone
0,1,Speedy Express,(503) 555-9831
1,2,United Package,(503) 555-3199
2,3,Federal Shipping,(503) 555-9931


In [16]:
df_time = pd.read_sql_query(
    """
SELECT distinct
    strftime('%Y%m%d',[order].ShippedDate ) AS TimeId,
    [order].ShippedDate as fecha,
        strftime('%Y', [order].ShippedDate) AS año,
       strftime('%m', [order].ShippedDate) AS mes,
       strftime('%d', [order].ShippedDate) AS dia,
       case strftime('%w',[order].ShippedDate)
         when '0' then 'Domingo'
         when '1' then 'Lunes'
         when '2' then 'Martes'
         when '3' then 'Miercoles'
         when '4' then 'Jueves'
         when '5' then 'Viernes'
         when '6' then 'Sabado'
       end as nombre_dia,
       strftime('%w', [order].ShippedDate) as dia_semana,
       strftime('%W', [order].ShippedDate) AS semana,
       case 
        when 0 + strftime('%m', [order].ShippedDate) between  1 and  4 then 'Primer Trimestre'
        when 0 + strftime('%m', [order].ShippedDate) between  5 and  8 then 'Segundo Trimestre'
        when 0 + strftime('%m', [order].ShippedDate) between  9 and  12 then 'Tercer Trimestre'
    end as trimestre     
FROM [order] 
where TimeId IS NOT NULL
group by TimeId;
    """
    ,con)
df_time.head(5)

,TimeId,fecha,año,mes,dia,nombre_dia,dia_semana,semana,trimestre
0,20120710,2012-07-10,2012,07,10,Martes,2,28,Segundo Trimestre
1,20120711,2012-07-11,2012,07,11,Miercoles,3,28,Segundo Trimestre
2,20120712,2012-07-12,2012,07,12,Jueves,4,28,Segundo Trimestre
3,20120713,2012-07-13 22:19:41,2012,07,13,Viernes,5,28,Segundo Trimestre
4,20120715,2012-07-15,2012,07,15,Domingo,0,28,Segundo Trimestre


In [17]:
df_fact = pd.read_sql_query(
    """
SELECT DISTINCT OrderDetail.Id AS Id_fact,
       [order].CustomerId AS id_customer,
       [order].ShipVia AS id_shipper,
       strftime('%Y%m%d', [order].ShippedDate) AS id_time,
       OrderDetail.ProductId as id_product,
       [order].Id as Id_location,
       employee.Id as Id_employe,
       [order].Freight,
       OrderDetail.UnitPrice,
       OrderDetail.Quantity,
       OrderDetail.Discount,
       round((((OrderDetail.UnitPrice*OrderDetail.Quantity) * (1-OrderDetail.Discount))/1)+[order].Freight,1) as finalprice
  FROM [order]
  inner join OrderDetail on [order].Id = OrderDetail.OrderId
  inner join employee on [order].EmployeeId = employee.Id;
    """
    ,con)
df_fact.head(5)

,Id_fact,id_customer,id_shipper,id_time,id_product,Id_location,Id_employe,Freight,UnitPrice,Quantity,Discount,finalprice
0,10248/11,VINET,3,20120716,11,10248,5,16.75,14.0,12,0.0,184.8
1,10248/42,VINET,3,20120716,42,10248,5,16.75,9.8,10,0.0,114.8
2,10248/72,VINET,3,20120716,72,10248,5,16.75,34.8,5,0.0,190.8
3,10249/14,TOMSP,1,20120710,14,10249,6,22.25,18.6,9,0.0,189.7
4,10249/51,TOMSP,1,20120710,51,10249,6,22.25,42.4,40,0.0,1718.3


In [18]:
df_product = pd.read_sql_query(
    """
SELECT product.Id AS Id_product,
       product.ProductName,
       supplier.Id AS Id_supplier,
       category.Id AS Id_Category,
       product.QuantityPerUnit,
       product.UnitPrice,
       product.UnitsInStock,
       product.UnitsOnOrder,
       product.ReorderLevel,
       product.Discontinued
  FROM product
       INNER JOIN
       category ON product.CategoryId = category.Id
       INNER JOIN
       supplier ON product.SupplierId = supplier.Id;
    """
    ,con)
df_product.head(5)

,Id_product,ProductName,Id_supplier,Id_Category,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1


In [19]:
df_supple = pd.read_sql_query(
    """
SELECT supplier.Id AS Id_supplier,
       supplier.CompanyName,
       supplier.ContactName,
       supplier.ContactTitle,
       supplier.Address,
       supplier.City,
       replace(supplier.Region,"NSW","New South Wales") as Region,
       supplier.PostalCode,
       replace(replace(supplier.Country,"UK","United Kingdom"),"USA","The United States of America") as Country,
       supplier.Phone,
       ifnull(supplier.Fax,'No Asignado') as Fax,
       ifnull(supplier.HomePage,'No asignado') as HomePage
  FROM supplier;
    """
    ,con)
df_supple.head(5)

,Id_supplier,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,HomePage
0,1,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,British Isles,EC1 4SD,United Kingdom,(171) 555-2222,No Asignado,No asignado
1,2,New Orleans Cajun Delights,Shelley Burke,Order Administrator,P.O. Box 78934,New Orleans,North America,70117,The United States of America,(100) 555-4822,No Asignado,#CAJUN.HTM#
2,3,Grandma Kelly's Homestead,Regina Murphy,Sales Representative,707 Oxford Rd.,Ann Arbor,North America,48104,The United States of America,(313) 555-5735,(313) 555-3349,No asignado
3,4,Tokyo Traders,Yoshi Nagase,Marketing Manager,9-8 Sekimai Musashino-shi,Tokyo,Eastern Asia,100,Japan,(03) 3555-5011,No Asignado,No asignado
4,5,Cooperativa de Quesos 'Las Cabras',Antonio del Valle Saavedra,Export Administrator,Calle del Rosal 4,Oviedo,Southern Europe,33007,Spain,(98) 598 76 54,No Asignado,No asignado


In [20]:
df_cate = pd.read_sql_query(
    """
SELECT category.Id AS Id_category,
       category.CategoryName,
       category.Description
  FROM category;

    """
    ,con)
df_cate.head(5)

,Id_category,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"


LOAD

In [9]:
con2 = sqlite3.connect("DW_SNOW.db")
con2

dim_time

In [10]:
df_time.to_sql('dim_Time',con2,if_exists='append',index=False)

1339

dim_customer

In [21]:
df_custom.to_sql('dim_customer', con2, if_exists='append',index=False)

91

dim_employee

In [22]:
df_employee.to_sql('dim_employee', con2, if_exists='append',index=False)

9

dim_shipper

In [23]:
df_shipper.to_sql('dim_shipper',con2,if_exists='append',index=False)

3

fact_sales

In [25]:
df_fact.to_sql('fact_sales',con2,if_exists='append',index=False)

621883

dim_product

In [24]:
df_product.to_sql('dim_product',con2,if_exists='append',index=False)

77

dim_Category

In [26]:
df_cate.to_sql('dim_category',con2,if_exists='append',index=False)

8

dim_supplier

In [27]:
df_supple.to_sql('dim_supplier',con2,if_exists='append',index=False)

29

dim_location

In [28]:
df_location.to_sql('dim_location',con2,if_exists='append',index=False)

16818